# Fine-Tuning
Harvard AI Bootcamp

## Make a copy of this notebook! Editing directly will not be saved.

In [ ]:
# Transformers installation
! pip install transformers==4.32.0 datasets

# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

# You may need to run the following two installs to fix a later error https://stackoverflow.com/questions/76448287/how-can-i-solve-importerror-using-the-trainer-with-pytorch-requires-accele
! pip install -U accelerate
! pip install -U transformers

  Using cached transformers-4.32.0-py3-none-any.whl.metadata (118 kB)
  Using cached tokenizers-0.13.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.32.0-py3-none-any.whl (7.5 MB)
Using cached tokenizers-0.13.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.4
    Uninstalling transformers-4.52.4:
      Successfully uninstalled transformers-4.52.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.32.0 which is incompatible.
  Using cached 

In [ ]:
! pip install --upgrade datasets fsspec

  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)


# Fine-tune a pretrained model

There are significant benefits to using a pretrained model. It reduces computation costs, your carbon footprint, and allows you to use state-of-the-art models without having to train one from scratch. Transformer provides access to thousands of pretrained models for a wide range of tasks. When you use a pretrained model, you train it on a dataset specific to your task. This is known as fine-tuning, an incredibly powerful training technique. In this tutorial, we will fine-tune a pretrained model with a deep learning framework.

## Prepare a dataset

Before you can fine-tune a pretrained model, download a dataset and prepare it for training.

In [ ]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

We need to tokenize our data so that it is truncated or padded into the same input length, in order for our data to be usable in any machine learning models that can only handle a fixed input windows. Given that this is time intensive, we can first create smaller datasets to use.

In [ ]:
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = dataset["test"].shuffle(seed=42).select(range(1000))

To process your dataset in one step, use Datasets map method to apply a preprocessing function over the entire dataset:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    # TODO: use a tokenizer on a examples["text"] and perform padding to max length and truncation
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True
    )

#Tokenize the datasets
small_train_dataset = small_train_dataset.map(tokenize_function, batched=True)
small_eval_dataset = small_eval_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

<a id='trainer'></a>

## Train

At this point, you should follow the section corresponding to the framework you want to use. You can use the links
in the right sidebar to jump to the one you want - and if you want to hide all of the content for a given framework,
just use the button at the top-right of that framework's block!

## Train with PyTorch Trainer


Start by loading your model and specify the number of expected labels. From the Yelp Review dataset card, you know there are five labels:

In [ ]:
from transformers import AutoModelForSequenceClassification
# TODO: modify below, set the number of labels of this model to 5
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<Tip>

You will see a warning about some of the pretrained weights not being used and some weights being randomly
initialized. Don't worry, this is completely normal! The pretrained head of the BERT model is discarded, and replaced with a randomly initialized classification head. You will fine-tune this new model head on your sequence classification task, transferring the knowledge of the pretrained model to it.

</Tip>

### Training hyperparameters

Next, create a TrainingArguments class which contains all the hyperparameters you can tune as well as flags for activating different training options. For this tutorial you can start with the default training hyperparameters, but feel free to experiment with these to find your optimal settings.

Specify where to save the checkpoints from your training:

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

### Evaluate

In [ ]:
import numpy as np
! pip install evaluate
import evaluate

metric = evaluate.load("accuracy")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.0 MB/s eta 0:00:00


Call `compute` on `metric` to calculate the accuracy of your predictions. Before passing your predictions to `compute`, you need to convert the predictions to logits

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

If you'd like to monitor your evaluation metrics during fine-tuning, specify the `evaluation_strategy` parameter in your training arguments to report the evaluation metric at the end of each epoch (for now we have 3 epochs, but if you have more time feel free to try a higher number):

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer",
                                  eval_strategy="epoch",
                                  logging_dir="./logs",
                                  logging_strategy="steps",
                                  logging_steps=10,
                                  report_to="none",
                                  num_train_epochs = 3) # Number of epochs (3 takes about 5 minutes with T4 GPU before evaluating)

### Trainer

Create a Trainer object with your model, training arguments, training and test datasets, and evaluation function

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    # TODO: Fill out the last 2 parameters with the datasets we created above
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset
)

Then fine-tune your model by calling train()

In [ ]:
trainer.train()

<a id='keras'></a>

In [22]:
import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

# find your exact notebook path
for root, _, files in os.walk('/content/drive/MyDrive'):
    for fn in files:
        if fn.endswith('.ipynb') and 'Day' in fn:
            print(os.path.join(root, fn))

# once you see the exact path, replace nb_path below:
nb_path = '/content/drive/MyDrive/Colab Notebooks/Day 2 - Fine Tuning HW.ipynb'

import json
with open(nb_path, 'r', encoding='utf-8') as f:
    nb = json.load(f)
nb.get('metadata', {}).pop('widgets', None)
for cell in nb.get('cells', []):
    cell.get('metadata', {}).pop('widgets', None)
with open(nb_path, 'w', encoding='utf-8') as f:
    json.dump(nb, f, indent=1)

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Copy of Day 1 - Neural Networks.ipynb
/content/drive/MyDrive/Colab Notebooks/Day 1 - Neural Networks.ipynb
/content/drive/MyDrive/Colab Notebooks/Day 2 - NLP Classification: RNNs and BERT Transformers.ipynb
/content/drive/MyDrive/Colab Notebooks/Day 2 - Mini LLM and Quantization.ipynb
/content/drive/MyDrive/Colab Notebooks/Day 2 - Fine Tuning HW.ipynb
